## STEP 01. Import Modules

In [3]:
import numpy as np
import pandas as pd
import json
from tqdm import *

import numpy as np
import pandas as pd
import re, nltk, spacy, gensim

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt

c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [2]:
import os
os.getcwd();

c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## STEP 02. Set Target csv File List

In [1]:
list = [ 'youtube_comments']

## STEP 03. PREPROCESSING - OPTION

In [4]:
# For user analysis, sometimes spamming and repetitve crawling results are what you need.
# So when you need those results, you can skip this STEP or do it in part.

for name in tqdm(list) :
    name_01 = name + ".csv"    
    df = pd.read_csv(name_01).dropna()
    df.reset_index(drop=True, inplace=True)
       
    print("삭제 대상을 확인 중입니다.")
    
    for i in tqdm(range(len(df))) :
        try :
            target_01 = len( [ x for x in df['title'] if x == df['title'][i] ])
        except :
            target_01 = len( [ x for x in df['user_text'] if x == df['user_text'][i] ])
            
        target_02 = len( [ y for y in df['url'] if y == df['url'][i] ])
        
        if target_01 > 1 :
            df['title'][i] = "delete target"
        elif target_02 > 1 :
            df['url'][i] = "delete target"
    
    target_list = []
    
    print("삭제 대상을 제거합니다.")
    
    before = len(df)
    
    for j in range(len(df)) :
        if df['title'][j] == "delete target" or df['url'][j] == "delete target" :
            target_list.append(j)
            
    df.drop(df.index[target_list], inplace=True)
    
    after = len(df)
    
    print(f"file name {name} 작업 완료...")
    print(f"총 {before - after}개의 열이 삭제되었습니다.")
    
    if os.path.isdir("./preprocessing") :
        pass
    else :
        os.mkdir("./preprocessing")
    
    df.to_csv(f"./preprocessing/{name_01}", index=False)
    print(f"파일 저장 완료...저장 위치 : ./preprocessing/{name_01}")
    

print("preprocessing 작업을 완료하였습니다.")

c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
  0%|                                                                                            | 0/1 [00:00<?, ?it/s]


삭제 대상을 확인 중입니다.


KeyError: 'url'

## STEP 04. Drawing LDA

In [11]:
df = pd.read_csv("news.csv")
df.columns = ['domain', 'keyword', 'user_text', 'channel', 'date']
df.to_csv("news.csv", index=False, encoding='utf-8-sig')

c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
df = pd.read_csv("news.csv")
df

c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,domain,keyword,uset_text,channel,date
0,NaverNews,이낙연,차기 대선 10개월 앞으로…농민단체 ‘농정의제’ 논의,농민신문,11시간 전
1,NaverNews,이낙연,후보따라 헤쳐모여...대선 잠룡 본격 '세대결',CJB청주방송,12시간 전
2,NaverNews,이낙연,"여론조사 업체 4곳 대권 적합도 조사 ‘이재명 25%, 윤석열 19%’… 尹 3주째 하락",세계일보,12시간 전
3,NaverNews,이낙연,"[나이트포커스] 이재명 ""윤석열, 예쁜 포장지밖에 못 봐서 내용 몰라""",YTN,12시간 전
4,NaverNews,이낙연,대권주자들의 5.18 방명록..말말말,kbc광주방송,13시간 전
...,...,...,...,...,...
10324,DaumNews,이재명,"'기본' 더하기 '성장'..與 왼날개 이재명, 중원行 출정식",아시아경제,2021.05.20
10325,DaumNews,이재명,"유시민 “대권 도전, 할 생각 없다”",일간투데이,2021.05.20
10326,DaumNews,이재명,'성공포럼' 창립식 기념촬영,뉴시스,2021.05.20
10327,DaumNews,이재명,'성공포럼' 창립식,뉴시스,2021.05.20


In [9]:
df.columns = ['domain', 'keyword', 'user_text', 'channel', 'date']
df.to_csv("news.csv", encoding='utf-8-sig', index=False)

c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
list = ['community']
min_df = 5
max_iteration = 2
NUM_TOPICS = 3

c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
name = 'twitter'
name_01 = name + ".csv"

df = pd.read_csv(name_01).dropna().drop_duplicates()
# Convert to list
data = df.user_text.values.tolist()

# user_text에서 명사만 추출 

from konlpy.tag import *
from nltk import Text

okt = Okt()

data_morph = []
for i in range(len(data)):

    try :
        data_morph.append(" ".join(okt.nouns(data[i])))
    except : # 중간에 비어있는 글자, na null 등 건너뛰기
#             print("scout 00 :", data[i])
        pass

data_morph

c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['추미애 젠더 의식 남자 판사 태권도 유단자 이상 폭력 범 앞 나 내 여성 정체 성은 우리 남편 알 의원님 상관 바',
 '추미애 젠더 의식 남자 판사 태권도 유단자 이상 폭력 범 앞 나 내 여성 정체 성은 우리 남편 알 의원님 상관 바',
 '차기 정부 거목',
 '차기 정부 거목',
 '민주당 파 충청 전북 일부 내각제 선호 정세균 경북 대구 일베 출신 좌꼴 연합 이재명 호남 서울 경기 기반 정통 이낙연 당신 선택',
 '대선 후보 정치 나이 이 경우 유권자 어필 고민 이해 하나 그 어필 방법 꼭 흉내내 신입사원 부장 포지션 해 움 어른 이미지 역시 유권자 어필 방법',
 '자긍심 나머지 좌절 감 트윗 인용 경향신문 민주당 집값 세금 부동산 불패 신화 다시 한번 확신 집 입자 다시 한번 좌절 배신',
 '자긍심 나머지 좌절 감 트윗 인용 경향신문 민주당 집값 세금 부동산 불패 신화 다시 한번 확신 집 입자 다시 한번 좌절 배신',
 '사람 세상 사람 나라 기억 바로 내일 오후 유튜브 추미애 의 출발 선 시오 오직 시민 여러분 깃발 지금 바로가기 구독 알람 설정',
 '오늘 추미애',
 '하위 공공 의료 제공 테 상위 민영 의료 이용 하위 공교육 제공 테 상위 사립학교 이용 생각 양극화 심화 공동체 붕괴',
 '목 정세균 전 국무총리 한국 노총 임원 간담 회 정세균 대한민국',
 '목 정세균 전 국무총리 한국 노총 임원 간담 회 정세균 대한민국',
 '목 정세균 전 국무총리 한국 노총 임원 간담 회 정세균 대한민국',
 '목 정세균 전 국무총리 한국 노총 임원 간담 회 정세균 대한민국',
 '목 정세균 전 국무총리 한국 노총 임원 간담 회 정세균 대한민국',
 '목 정세균 전 국무총리 한국 노총 임원 간담 회 정세균 대한민국',
 '목 정세균 전 국무총리 한국 노총 임원 간담 회 정세균 대한민국',
 '이낙연 캠프 대변인 국회의원 영훈 논평 공감 올해 이낙연 전 대표 주변 인사 대한 허위 사실 지속 허위 사실 유포 행위 중단 사과 이낙연 전 대표 측 요구 

In [20]:

vectorizer = CountVectorizer(analyzer='word',       
                         min_df=min_df,   # 특정 횟수 이상 언급된 것만 따로                      
                         max_features=50000,             
                        )

data_vectorized = vectorizer.fit_transform(data_morph)
print(data_vectorized)

c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


  (0, 3919)	1
  (0, 3494)	1
  (0, 2941)	1
  (0, 734)	1
  (0, 4164)	1
  (0, 2990)	1
  (0, 4244)	1
  (0, 2580)	1
  (0, 3451)	1
  (0, 2088)	1
  (0, 2774)	1
  (0, 737)	1
  (0, 2945)	1
  (0, 1962)	1
  (1, 3919)	1
  (1, 3494)	1
  (1, 2941)	1
  (1, 734)	1
  (1, 4164)	1
  (1, 2990)	1
  (1, 4244)	1
  (1, 2580)	1
  (1, 3451)	1
  (1, 2088)	1
  (1, 2774)	1
  :	:
  (14434, 3046)	1
  (14434, 4241)	1
  (14434, 3987)	1
  (14434, 3338)	2
  (14434, 3448)	1
  (14435, 3919)	1
  (14436, 3016)	1
  (14436, 2022)	1
  (14436, 938)	2
  (14436, 2904)	1
  (14436, 3942)	2
  (14436, 2053)	2
  (14436, 3123)	1
  (14436, 3579)	1
  (14436, 3670)	1
  (14436, 320)	1
  (14436, 2656)	1
  (14436, 1569)	1
  (14436, 4092)	1
  (14436, 4463)	1
  (14436, 1838)	1
  (14436, 2901)	1
  (14436, 4091)	1
  (14436, 1068)	1
  (14436, 3004)	1


In [21]:
df = pd.DataFrame(data=data_vectorized)
df

c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0
0,"(0, 3919)\t1\n (0, 3494)\t1\n (0, 2941)\t1..."
1,"(0, 3919)\t1\n (0, 3494)\t1\n (0, 2941)\t1..."
2,"(0, 3776)\t1\n (0, 3416)\t1"
3,"(0, 3776)\t1\n (0, 3416)\t1"
4,"(0, 1464)\t1\n (0, 3962)\t1\n (0, 3348)\t1..."
...,...
14432,"(0, 3919)\t1\n (0, 1464)\t3\n (0, 2968)\t1..."
14433,"(0, 3919)\t1\n (0, 2900)\t1\n (0, 4192)\t1"
14434,"(0, 3919)\t1\n (0, 3776)\t1\n (0, 1464)\t1..."
14435,"(0, 3919)\t1"


In [16]:
print(data_vectorized.shape)

(14437, 4597)


c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:

vectorizer = HashingVectorizer(analyzer='word',    
                               n_features=50000
                        )

data_vectorized = vectorizer.fit_transform(data_morph)
print(data_vectorized.shape)
print(data_vectorized)

c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(14437, 50000)
  (0, 5935)	-0.25
  (0, 8008)	0.25
  (0, 8897)	0.25
  (0, 10211)	0.25
  (0, 12841)	-0.25
  (0, 21108)	0.25
  (0, 22381)	-0.25
  (0, 26317)	0.25
  (0, 26683)	-0.25
  (0, 29419)	-0.25
  (0, 30558)	-0.25
  (0, 39567)	-0.25
  (0, 40152)	-0.25
  (0, 41956)	-0.25
  (0, 42532)	0.25
  (0, 49110)	0.25
  (1, 5935)	-0.25
  (1, 8008)	0.25
  (1, 8897)	0.25
  (1, 10211)	0.25
  (1, 12841)	-0.25
  (1, 21108)	0.25
  (1, 22381)	-0.25
  (1, 26317)	0.25
  (1, 26683)	-0.25
  :	:
  (14434, 35487)	0.20851441405707477
  (14434, 36243)	0.20851441405707477
  (14434, 37618)	-0.20851441405707477
  (14434, 40599)	-0.41702882811414954
  (14435, 5935)	-1.0
  (14436, 2633)	0.17677669529663687
  (14436, 8273)	0.17677669529663687
  (14436, 9632)	0.17677669529663687
  (14436, 9955)	0.17677669529663687
  (14436, 21205)	0.17677669529663687
  (14436, 21215)	-0.17677669529663687
  (14436, 21407)	-0.17677669529663687
  (14436, 21679)	0.17677669529663687
  (14436, 21771)	0.17677669529663687
  (14436, 24569)	-0.

In [1]:
for name in list :
    name_01 = name + ".csv"
    
    df = pd.read_csv(name_01).dropna()
    # Convert to list
    data = df.user_text.values.tolist()

    # user_text에서 명사만 추출 

    from konlpy.tag import *
    from nltk import Text

    okt = Okt()

    data_morph = []
    for i in range(len(data)):

        try :
            data_morph.append(" ".join(okt.nouns(data[i])))
        except : # 중간에 비어있는 글자, na null 등 건너뛰기
#             print("scout 00 :", data[i])
            pass


    vectorizer = CountVectorizer(analyzer='word',       
                             min_df=min_df,   # 특정 횟수 이상 언급된 것만 따로                      
                             max_features=50000,             
                            )

    data_vectorized = vectorizer.fit_transform(data_morph)


    NUM_TOPICS = NUM_TOPICS # 분류하는 카테고리 숫자
    lda_model = LatentDirichletAllocation(
                            n_components=NUM_TOPICS,              
                            max_iter=max_iteration,               
                            learning_method='online',   
                            random_state=100,          
                            batch_size=128,            
                            evaluate_every = -1,       
                            #n_jobs = -1,               
                        )
    lda_output = lda_model.fit_transform(data_vectorized)
    log = lda_model.score(data_vectorized)
    per = lda_model.perplexity(data_vectorized)
    
    # Create Document - Topic Matrix
    lda_output = lda_model.transform(data_vectorized)

    # column names
    topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

    # index names
    docnames = ["Doc" + str(i) for i in range(len(data))]

    # Make the pandas dataframe
    df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['dominant_topic'] = dominant_topic

    # Styling
    def color_green(val):
        color = 'green' if val > .1 else 'black'
        return 'color: {col}'.format(col=color)

    def make_bold(val):
        weight = 700 if val > .1 else 400
        return 'font-weight: {weight}'.format(weight=weight)

    # Apply Style
    df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
    
    # Create Document - Topic Matrix
    lda_output = lda_model.transform(data_vectorized)

    # column names
    topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

    # index names
    docnames = ["Doc" + str(i) for i in range(len(data))]

    # Make the pandas dataframe
    df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['dominant_topic'] = dominant_topic

    # Styling
    def color_green(val):
        color = 'green' if val > .1 else 'black'
        return 'color: {col}'.format(col=color)

    def make_bold(val):
        weight = 700 if val > .1 else 400
        return 'font-weight: {weight}'.format(weight=weight)

    # Apply Style
    df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
    print(df_document_topics)
    
    name_02 = name + "_reference" + ".html"
        
    pyLDAvis.enable_notebook()
    p = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)
    
    pyLDAvis.save_html(p,name_02)
    
        # Show top n keywords for each topic
    def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
        keywords = np.array(vectorizer.get_feature_names())
        topic_keywords = []
        for topic_weights in lda_model.components_:
            top_keyword_locs = (-topic_weights).argsort()[:n_words]
            topic_keywords.append(keywords.take(top_keyword_locs))
        return topic_keywords

    topic_keywords = show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=11) 
    
    name_03 = name + "_reference" +".csv"
    pd.DataFrame(topic_keywords).to_csv(name_03, index=False, encoding='utf-8-sig')
    print(pd.DataFrame(topic_keywords));

TypeError: 'type' object is not iterable

In [ ]:
###### pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)

In [6]:
for name in list :
    name_01 = name + ".csv"
    keyword = "이재명"
    
    df = pd.read_csv(name_01).dropna()
    df2 = df[df['keyword'] == keyword]
    # Convert to list
    data = df2.user_text.values.tolist()

    # user text에서 명사만 추출 

    from konlpy.tag import *
    from nltk import Text

    okt = Okt()

    data_morph = []
    for i in range(len(data)):
        data_morph.append(" ".join(okt.nouns(data[i])))

    vectorizer = CountVectorizer(analyzer='word',       
                             min_df=min_df,   # 특정 횟수 이상 언급된 것만 따로                      
                             max_features=50000,             
                            )

    data_vectorized = vectorizer.fit_transform(data_morph)


    NUM_TOPICS = NUM_TOPICS # 분류하는 카테고리 숫자
    lda_model = LatentDirichletAllocation(
                            n_components=NUM_TOPICS,              
                            max_iter=max_iteration,               
                            learning_method='online',   
                            random_state=100,          
                            batch_size=128,            
                            evaluate_every = -1,       
                            #n_jobs = -1,               
                        )
    lda_output = lda_model.fit_transform(data_vectorized)
    log = lda_model.score(data_vectorized)
    per = lda_model.perplexity(data_vectorized)
    
    # Create Document - Topic Matrix
    lda_output = lda_model.transform(data_vectorized)

    # column names
    topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

    # index names
    docnames = ["Doc" + str(i) for i in range(len(data))]

    # Make the pandas dataframe
    df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['dominant_topic'] = dominant_topic

    # Styling
    def color_green(val):
        color = 'green' if val > .1 else 'black'
        return 'color: {col}'.format(col=color)

    def make_bold(val):
        weight = 700 if val > .1 else 400
        return 'font-weight: {weight}'.format(weight=weight)

    # Apply Style
    df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
    
    # Create Document - Topic Matrix
    lda_output = lda_model.transform(data_vectorized)

    # column names
    topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

    # index names
    docnames = ["Doc" + str(i) for i in range(len(data))]

    # Make the pandas dataframe
    df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['dominant_topic'] = dominant_topic

    # Styling
    def color_green(val):
        color = 'green' if val > .1 else 'black'
        return 'color: {col}'.format(col=color)

    def make_bold(val):
        weight = 700 if val > .1 else 400
        return 'font-weight: {weight}'.format(weight=weight)

    # Apply Style
    df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
    print(df_document_topics)
    
    name_02 = keyword + "_reference" + ".html"
        
    pyLDAvis.enable_notebook()
    p = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)
    
    pyLDAvis.save_html(p,name_02)
    
        # Show top n keywords for each topic
    def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
        keywords = np.array(vectorizer.get_feature_names())
        topic_keywords = []
        for topic_weights in lda_model.components_:
            top_keyword_locs = (-topic_weights).argsort()[:n_words]
            topic_keywords.append(keywords.take(top_keyword_locs))
        return topic_keywords

    topic_keywords = show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=11) 
    
    name_03 = keyword +  "_reference" +".csv"
    pd.DataFrame(topic_keywords).to_csv(name_03, index=False, encoding='utf-8-sig')
    print(pd.DataFrame(topic_keywords));

pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)

c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


    0    1    2   3    4    5   6   7    8    9    10
0  민주당   경선   진짜  정말  지지자  이낙연  자기  검찰   지금   소리   이제
1   사람   생각  이재명  정치   국민  민주당  지지  문제   사면   이번   중도
2  이재명  대통령  민주당  대선   지사  윤석열  의원  후보  경기도  문재인  지지율


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.161855  0.193519       1        1  46.571332
1     -0.123058 -0.211922       2        1  29.710630
0      0.284913  0.018403       3        1  23.718038, topic_info=   Term        Freq       Total Category  logprob  loglift
22   사람  253.000000  253.000000  Default  30.0000  30.0000
34  이재명  671.000000  671.000000  Default  29.0000  29.0000
25   생각  118.000000  118.000000  Default  28.0000  28.0000
10  대통령  166.000000  166.000000  Default  27.0000  27.0000
3    경선  106.000000  106.000000  Default  26.0000  26.0000
..  ...         ...         ...      ...      ...      ...
16  문재인    1.386834   60.784457   Topic3  -6.6320  -2.3414
30  윤석열    2.171963   97.578580   Topic3  -6.1834  -2.3661
34  이재명   13.145019  671.862534   Topic3  -4.3830  -2.4951
41   정치    1.570503   97.780278   Topic3  -6.5076  -2.6924
22   사람    2.187232  253.578746   Topic3  -6.1764  -3.3141

[126 rows x 6 columns], token_table=      Topic      Freq Term
term                      
0         1  0.018521   검찰
0         2  0.018521   검찰
0         3  0.981595   검찰
1         1  0.962794   결과
1         2  0.030087   결과
...     ...       ...  ...
52        3  0.812588   하나
53        2  0.970840   해도
54        1  0.757042   후보
54        2  0.232936   후보
54        3  0.009706   후보

[140 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1])

In [7]:
for name in list :
    name_01 = name + ".csv"
    keyword = "이낙연"
    
    df = pd.read_csv(name_01).dropna()
    df2 = df[df['keyword'] == keyword]
    # Convert to list
    data = df2.user_text.values.tolist()

    # user text에서 명사만 추출 

    from konlpy.tag import *
    from nltk import Text

    okt = Okt()

    data_morph = []
    for i in range(len(data)):
        data_morph.append(" ".join(okt.nouns(data[i])))

    vectorizer = CountVectorizer(analyzer='word',       
                             min_df=min_df,   # 특정 횟수 이상 언급된 것만 따로                      
                             max_features=50000,             
                            )

    data_vectorized = vectorizer.fit_transform(data_morph)


    NUM_TOPICS = NUM_TOPICS # 분류하는 카테고리 숫자
    lda_model = LatentDirichletAllocation(
                            n_components=NUM_TOPICS,              
                            max_iter=max_iteration,               
                            learning_method='online',   
                            random_state=100,          
                            batch_size=128,            
                            evaluate_every = -1,       
                            #n_jobs = -1,               
                        )
    lda_output = lda_model.fit_transform(data_vectorized)
    log = lda_model.score(data_vectorized)
    per = lda_model.perplexity(data_vectorized)
    
    # Create Document - Topic Matrix
    lda_output = lda_model.transform(data_vectorized)

    # column names
    topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

    # index names
    docnames = ["Doc" + str(i) for i in range(len(data))]

    # Make the pandas dataframe
    df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['dominant_topic'] = dominant_topic

    # Styling
    def color_green(val):
        color = 'green' if val > .1 else 'black'
        return 'color: {col}'.format(col=color)

    def make_bold(val):
        weight = 700 if val > .1 else 400
        return 'font-weight: {weight}'.format(weight=weight)

    # Apply Style
    df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
    
    # Create Document - Topic Matrix
    lda_output = lda_model.transform(data_vectorized)

    # column names
    topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

    # index names
    docnames = ["Doc" + str(i) for i in range(len(data))]

    # Make the pandas dataframe
    df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['dominant_topic'] = dominant_topic

    # Styling
    def color_green(val):
        color = 'green' if val > .1 else 'black'
        return 'color: {col}'.format(col=color)

    def make_bold(val):
        weight = 700 if val > .1 else 400
        return 'font-weight: {weight}'.format(weight=weight)

    # Apply Style
    df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
    print(df_document_topics)
    
    name_02 = keyword + "_reference" + ".html"
        
    pyLDAvis.enable_notebook()
    p = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)
    
    pyLDAvis.save_html(p,name_02)
    
        # Show top n keywords for each topic
    def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
        keywords = np.array(vectorizer.get_feature_names())
        topic_keywords = []
        for topic_weights in lda_model.components_:
            top_keyword_locs = (-topic_weights).argsort()[:n_words]
            topic_keywords.append(keywords.take(top_keyword_locs))
        return topic_keywords

    topic_keywords = show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=11) 
    
    name_03 = keyword +  "_reference" +".csv"
    pd.DataFrame(topic_keywords).to_csv(name_03, index=False, encoding='utf-8-sig')
    print(pd.DataFrame(topic_keywords));

pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)

c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


    0    1   2    3    4   5    6    7    8   9    10
0  이재명  민주당  대선  이낙연   그냥  이제   하나   자기   사과  생각  지지율
1  이낙연   생각  사면   지금  지지율  진짜   정도   본인  대통령  자기   그냥
2   사람   대표  사과   국민   정치  자기  민주당  대통령  이낙연  생각   정도


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.246961 -0.152137       1        1  36.675114
2     -0.272434 -0.113110       2        1  33.627516
0      0.025474  0.265247       3        1  29.697370, topic_info=   Term        Freq       Total Category  logprob  loglift
12  이재명  115.000000  115.000000  Default  21.0000  21.0000
4    대표  115.000000  115.000000  Default  20.0000  20.0000
8    사람  120.000000  120.000000  Default  19.0000  19.0000
5   민주당  126.000000  126.000000  Default  18.0000  18.0000
7    사과   92.000000   92.000000  Default  17.0000  17.0000
..  ...         ...         ...      ...      ...      ...
17   지금    0.870163   50.329745   Topic3  -6.2241  -2.8436
0    국민    0.793540   47.195689   Topic3  -6.3163  -2.8714
19   진짜    0.525370   36.630104   Topic3  -6.7287  -3.0304
4    대표    1.117522  115.250369   Topic3  -5.9739  -3.4219
9    사면    0.478565   70.832985   Topic3  -6.8220  -3.7832

[84 rows x 6 columns], token_table=      Topic      Freq Term
term                      
0         2  0.974665   국민
0         3  0.021188   국민
1         1  0.115600   그냥
1         2  0.069360   그냥
1         3  0.809198   그냥
2         3  0.969737   대선
3         1  0.636193  대통령
3         2  0.348880  대통령
3         3  0.041045  대통령
4         1  0.008677   대표
4         2  0.980474   대표
4         3  0.008677   대표
5         1  0.007889  민주당
5         2  0.173550  민주당
5         3  0.820419  민주당
6         1  0.934684   본인
6         2  0.029209   본인
6         3  0.029209   본인
7         1  0.010803   사과
7         2  0.939896   사과
7         3  0.054017   사과
8         1  0.008284   사람
8         2  0.969241   사람
8         3  0.024852   사람
9         1  0.974122   사면
9         2  0.014118   사면
10        1  0.901560   생각
10        2  0.052416   생각
10        3  0.041933   생각
11        1  0.759730  이낙연
11        2  0.036178  이낙연
11        3  0.201561  이낙연
12        1  0.008643  이재명
12        3  0.985269  이재명
13        1  0.100104   이제
13        2  0.050052   이제
13        3  0.850884   이제
14        1  0.121887   자기
14        2  0.629749   자기
14        3  0.243774   자기
15        1  0.861091   정도
15        2  0.126631   정도
15        3  0.025326   정도
16        1  0.026873   정치
16        2  0.967440   정치
16        3  0.026873   정치
17        1  0.953710   지금
17        2  0.039738   지금
17        3  0.019869   지금
18        1  0.873876  지지율
18        2  0.045993  지지율
18        3  0.068990  지지율
19        1  0.982798   진짜
19        3  0.027300   진짜
20        3  0.964718   하나, R=21, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])

In [3]:
for name in list :
    name_01 = name + ".csv"
    keyword = "정세균"
    
    df = pd.read_csv(name_01).dropna()
    df2 = df[df['keyword'] == keyword]
    # Convert to list
    data = df2.user_text.values.tolist()

    # user text에서 명사만 추출 

    from konlpy.tag import *
    from nltk import Text

    okt = Okt()

    data_morph = []
    for i in range(len(data)):
        data_morph.append(" ".join(okt.nouns(data[i])))

    vectorizer = CountVectorizer(analyzer='word',       
                             min_df=min_df,   # 특정 횟수 이상 언급된 것만 따로                      
                             max_features=50000,             
                            )

    data_vectorized = vectorizer.fit_transform(data_morph)


    NUM_TOPICS = NUM_TOPICS # 분류하는 카테고리 숫자
    lda_model = LatentDirichletAllocation(
                            n_components=NUM_TOPICS,              
                            max_iter=max_iteration,               
                            learning_method='online',   
                            random_state=100,          
                            batch_size=128,            
                            evaluate_every = -1,       
                            #n_jobs = -1,               
                        )
    lda_output = lda_model.fit_transform(data_vectorized)
    log = lda_model.score(data_vectorized)
    per = lda_model.perplexity(data_vectorized)
    
    # Create Document - Topic Matrix
    lda_output = lda_model.transform(data_vectorized)

    # column names
    topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

    # index names
    docnames = ["Doc" + str(i) for i in range(len(data))]

    # Make the pandas dataframe
    df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['dominant_topic'] = dominant_topic

    # Styling
    def color_green(val):
        color = 'green' if val > .1 else 'black'
        return 'color: {col}'.format(col=color)

    def make_bold(val):
        weight = 700 if val > .1 else 400
        return 'font-weight: {weight}'.format(weight=weight)

    # Apply Style
    df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
    
    # Create Document - Topic Matrix
    lda_output = lda_model.transform(data_vectorized)

    # column names
    topicnames = ["Topic" + str(i) for i in range(lda_model.n_components)]

    # index names
    docnames = ["Doc" + str(i) for i in range(len(data))]

    # Make the pandas dataframe
    df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['dominant_topic'] = dominant_topic

    # Styling
    def color_green(val):
        color = 'green' if val > .1 else 'black'
        return 'color: {col}'.format(col=color)

    def make_bold(val):
        weight = 700 if val > .1 else 400
        return 'font-weight: {weight}'.format(weight=weight)

    # Apply Style
    df_document_topics = df_document_topic.head(15).style.applymap(color_green).applymap(make_bold)
    print(df_document_topics)
    
    name_02 = keyword + "_reference" + ".html"
        
    pyLDAvis.enable_notebook()
    p = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)
    
    pyLDAvis.save_html(p,name_02)
    
        # Show top n keywords for each topic
    def show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=20):
        keywords = np.array(vectorizer.get_feature_names())
        topic_keywords = []
        for topic_weights in lda_model.components_:
            top_keyword_locs = (-topic_weights).argsort()[:n_words]
            topic_keywords.append(keywords.take(top_keyword_locs))
        return topic_keywords

    topic_keywords = show_topics(vectorizer=vectorizer, lda_model=lda_model, n_words=11) 
    
    name_03 = keyword +  "_reference" +".csv"
    pd.DataFrame(topic_keywords).to_csv(name_03, index=False, encoding='utf-8-sig')
    print(pd.DataFrame(topic_keywords));

pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer)

c:\python38\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


   0    1   2    3    4    5    6   7    8   9   10
0  검찰   개혁  총리  정세균   언론   광주   대선  정신  노무현  사람  생각
1  지금  이재명  이제   대선   얘기  추미애  민주당  장관   조국  행보  선거
2  정치   후보  사람   행동  민주화   진짜   소리  정권   자리  대선  그냥


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.133469  0.045579       1        1  47.793439
1     -0.015863 -0.112478       2        1  29.667422
2     -0.117605  0.066899       3        1  22.539140, topic_info=   Term       Freq      Total Category  logprob  loglift
78   지금  26.000000  26.000000  Default  30.0000  30.0000
62  이재명  30.000000  30.000000  Default  29.0000  29.0000
4    검찰  56.000000  56.000000  Default  28.0000  28.0000
3    개혁  43.000000  43.000000  Default  27.0000  27.0000
74   정치  18.000000  18.000000  Default  26.0000  26.0000
..  ...        ...        ...      ...      ...      ...
18   대해   2.088872   6.289838   Topic3  -4.8652   0.3876
73   정신   7.358673  22.416746   Topic3  -3.6059   0.3760
15   대선   7.610417  35.635757   Topic3  -3.5723  -0.0539
65   자기   3.004581   9.665275   Topic3  -4.5017   0.3215
25  민주당   4.272213  21.819530   Topic3  -4.1497  -0.1408

[140 rows x 6 columns], token_table=      Topic      Freq Term
term                      
0         1  0.163943   가요
0         2  0.163943   가요
0         3  0.655771   가요
1         1  0.256691   가장
1         2  0.641728   가장
...     ...       ...  ...
89        2  0.752795   행보
89        3  0.107542   행보
90        1  0.214082   후보
90        2  0.256899   후보
90        3  0.556614   후보

[270 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3])